<a href="https://colab.research.google.com/github/zavdr/NeuroScan/blob/main/Brain_Tumour_MRI_Detection_Deep_Learning_Project_Zaviar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [4]:
# count the number of images in the classes (0 - Brain Tumour, 1 = Healthy)
ROOT_DIR = "/content/Brain Tumor Data Set/Brain Tumor Data Set"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

number_of_images.items()

dict_items([('Healthy', 16), ('Brain Tumor', 243)])

# i will now split the data such that:
- 70% for training data
- 15% for validation
- 15% for testing


In [5]:
# i will create a train folder

if not os.path.exists("./train"):
  os.mkdir("./train")

  for dir in os.listdir(ROOT_DIR):
    os.makedirs("./train/"+dir)

    for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)),
                                size = (math.floor(0.7*number_of_images[dir])-5),
                                replace = False):
      O = os.path.join(ROOT_DIR,dir,img)
      D = os.path.join("./train",dir)
      shutil.copy(O,D)
      os.remove(O)
else:
  print("the folder already exists")

the folder already exists


In [6]:
def dataFolder(path, split):
  if not os.path.exists("./"+path):
    os.mkdir("./"+path)

    for dir in os.listdir(ROOT_DIR):
      os.makedirs("./"+path+"/"+dir)

      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)),
                                size = (math.floor(split*number_of_images[dir])-5),
                                replace = False):
        O = os.path.join(ROOT_DIR,dir,img)
        D = os.path.join("./"+path,dir)
        shutil.copy(O,D)
        os.remove(O)
  else:
    print("the folder already exists")

In [7]:
dataFolder("train", 0.7)

the folder already exists


In [8]:
dataFolder("val", 0.15)

the folder already exists


In [9]:
dataFolder("test", 0.15)

the folder already exists


# MODEL BUILD

In [10]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras

In [11]:
# Convulation Neural Network (CNN) Model

model = Sequential()
model.add(Conv2D(filters=16, kernel_size= (3, 3), activation='relu', input_shape = (224,224,3), ))

model.add(Conv2D(filters=36, kernel_size= (3, 3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size= (3, 3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size= (3, 3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 220, 220, 36)   │         5,220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 36)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 108, 108, 64)   │        20,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,537,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,638,245 (21.51 MB)

 Trainable params: 5,638,245 (21.51 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Preparing my data using Data Generator

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input

In [14]:
def preprocessingImages1(path):
  """
  input : Path
  output : Preprocessed images
  """
  image_data = ImageDataGenerator(
      rescale = 1./255,
      shear_range = 0.2,
      preprocessing_function= keras.applications.mobilenet.preprocess_input,
      horizontal_flip = True
  )
  image = image_data.flow_from_directory(
      directory = path,
      target_size = (224,224),
      batch_size = 32,
      class_mode = 'binary'
  )
  return image

In [15]:
path = "/content/train"
train_data = preprocessingImages1(path)

Found 3148 images belonging to 2 classes.


In [16]:
train_data.class_indices

{'Brain Tumor': 0, 'Healthy': 1}

In [17]:
def preprocessingImages2(path):
  """
  input : Path
  output : Preprocessed images
  """
  image_data = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input)
  image = image_data.flow_from_directory(
      directory = path,
      target_size = (224,224),
      batch_size = 32,
      class_mode = 'binary'
  )
  return image

In [18]:
test_data = preprocessingImages2("/content/test")

Found 667 images belonging to 2 classes.


In [19]:
val_data = preprocessingImages2("/content/val")

Found 667 images belonging to 2 classes.


In [20]:
# early stopping and model check point

from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, mode = 'auto')

# model check point
mc  = ModelCheckpoint(monitor='val_accuracy', filepath="./bestmodel.h5",
                      verbose= 1 , save_best_only= True, mode = 'auto')

cd = [es,mc]

# MODEL TRAINING

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet
import keras

In [22]:
base_model = MobileNet(input_shape=(224,224,3), include_top=False)

In [23]:
for layer in base_model.layers:
  layer.trainable = False

In [24]:
X = Flatten()(base_model.output)
X = Dense(units=1, activation='sigmoid')(X)

model = Model(base_model.input, X)

In [25]:
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,279,041 (12.51 MB)

 Trainable params: 50,177 (196.00 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [26]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
## call back

from keras.callbacks import ModelCheckpoint, EarlyStopping

# model checkpoint
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1)

cb = [mc,es]

In [28]:
hist = model.fit(train_data,
          steps_per_epoch=8,
          epochs=30,
          validation_data=val_data,
          validation_steps=16,
          callbacks=cd)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.5708 - loss: 2.1384
Epoch 1: val_accuracy improved from -inf to 0.54883, saving model to ./bestmodel.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 980ms/step - accuracy: 0.5673 - loss: 2.1136 - val_accuracy: 0.5488 - val_loss: 3.0177
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.5446 - loss: 1.3533
Epoch 2: val_accuracy did not improve from 0.54883
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5420 - loss: 1.3405 - val_accuracy: 0.5098 - val_loss: 1.1756
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.5995 - loss: 0.8384
Epoch 3: val_accuracy did not improve from 0.54883
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 711ms/step - accuracy: 0.5950 - loss: 0.8520 - val_accuracy: 0.5215 - val_loss: 1.0980
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5628 - loss: 0.9740
Epoch 4: val_accuracy did not improve from 0.54883
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 224ms/step - accuracy: 0.5671 - loss: 0.9545 - val_accuracy: 0.4785 - val_loss: 1.3152
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.5968 - loss: 1.0159
Epoch 5: val_accuracy did not improve from 0.54883
8/8

In [29]:
# load the best fit model

model = load_model("/content/bestmodel.h5")

In [30]:
acc = model.evaluate(test_data)[1]
print(f"The accuracy of the model is {acc*100} %")

21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 199ms/step - accuracy: 0.5316 - loss: 3.1213
The accuracy of the model is 53.823089599609375 %


In [31]:
# model graphical interpretation!

h = hs.history
h.keys()

NameError: name 'hs' is not defined

In [32]:
import matplotlib.pyplot as plt
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")
plt.title("accuracy vs validation-accuracy")
plt.show()

NameError: name 'h' is not defined

In [33]:
import matplotlib.pyplot as plt
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")
plt.title("loss vs validation-loss")
plt.show()

NameError: name 'h' is not defined

In [34]:
# calculating model accuracy
from keras.models import load_model

model = load_model("/content/bestmodel.h5")

# MODEL ACCURACY

In [35]:
acc = model.evaluate(test_data)[1]
print(f"The accuracy of the model is {acc*100} %")

21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.5409 - loss: 2.9406
The accuracy of the model is 53.823089599609375 %


In [36]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [37]:
path = "/content/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (954).jpg"
img = load_img(path, target_size=(224,224))
i = img_to_array(img)/255
input_arr = np.array([i])
input_arr.shape

pred = np.argmax(model.predict(input_arr))

if round(pred) == 1:
    print("The MRI displays a Brain Tumour!")
else:
    print("The MRI does not display a Brain Tumour..")

# to display the image
plt.imshow(input_arr[0])
plt.title("Input image")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (954).jpg'

In [38]:
train_data.class_indices

{'Brain Tumor': 0, 'Healthy': 1}